- Parse the Economist web-site on topics by the keywords
https://www.economist.com/

In [1]:
from bs4 import BeautifulSoup as bs
import requests

# Write a class that does parsing:

Hi!
I've created a class that parser through the Economist web-site, gathers, provides links and downloads (the last 2 are optional).

I'm just now starting my IT path and i can see that this code is far from perfect but i'm working hard on it.

P.S. I am not sure if this project can be of any practical use, i've made it for fun and to exercise myself.
Peace!

# Ideas!

- browsing through the sections
    - shows what sections are available
- most trending kwords
- DATE,DATE,DATE

In [330]:
class economist_parser:
    
    def __init__(self, root, section, init_page, end_page, **kwargs):
        self.root = root
        self.section = section
        self.init_page = init_page
        self.end_page = end_page
        self.sep_sign = '='
        kwargs['root'] = self.root
        kwargs['section'] = self.section

    def status_checker (self):
        """
        Checking the status of access 
        """
        url = f'{self.root}/{self.section}'
        web_page = requests.get(url)
        status = web_page.status_code
        if status == 200:
            print(f'Your status code is: {status}. That means access granted, now can get to parsing')
        else:
            print('Sorry, there\'s some problems with access, try to parse the other way')
            
    def kw_definer(self):
        """
        Forms a list of keywords to check from articles
        Returns a list of keywords to iterate from
        """
        keywords = input('Please enter the keywords, dividing them by a single comma: \n')
        self.list_of_kws = [i.strip().lower() for i in keywords.split(',')]
        return self.list_of_kws
             
    def article_downloader(self, head, ref):
        """
        On a selected page, chooses adequate articles, accesses them and downloads the contents
        """
        import os.path
#             getting a link and accessing the page
        soup = bs(requests.get(f'{self.root}{ref}').text, 'lxml')
        main_text = soup.find_all('p', class_='article__body-text')
#             sacking the text of the article
        transcript = ''
        for n in range(len(main_text)):
            transcript += f'\t{main_text[n].get_text()} \n\n'
#             deleting chars that won't fit the name
        badchars = ['/', '?', '\\', '*']
        for c in badchars:
            head = head.replace(c, '')
#             loading files
        with open(f'{head}.docx', 'w', encoding='utf-8') as file:
            file.write(transcript)
#         checking
        if os.path.isfile(f'{head}.docx'):
            print('*'*20, 'Download success', '*'*20)
        else:
            print('Smth went wrong with downloading')
                  
    def parser_engine (self):
        """
        Selecting a page to parse on and accessing it;
        Selecting a body of the page with content and returning it
        """
        page_link = f'{self.root}/{self.section}?page={self.n_page}'
        soup = bs(requests.get(page_link).text, 'lxml')
        content_body = soup.find('main', role='main')
        self.page_parser(content_body)
        
    def page_parser (self, content_body, *args):
        """
        Parsing mechanism; 3 main stages
        1) Selecting a frame of the article and iterating through the frames; defining tags of different contents
        2) Iterating through the list of kwords, checking and matching
        3) Additional stuff - totaling, printing and returning results
        """
        self.counter = 0
        self.controller = []
        all_articles = content_body.find_all('div', class_='css-e6sfh4 e1mrg8dy0')
        for i in range(len(all_articles)): # check if the description is not empty. Is it's not - parses
            if all_articles[i].find('p', class_='css-1tj7b79 e1dj3j7p0'): #creating tags of parts of frames
                head = content_body.find_all('h3')[i].get_text()
                descr = content_body.find_all('p', class_='css-1tj7b79 e1dj3j7p0')[i].get_text()
                ref = content_body.find_all('a')[i]['href']
                self.selector(head, descr, ref)
        
        self.total_counter += self.counter
        
        if self.counter == 1:
            print(f'{self.sep_sign * 90}\n On the page {self.n_page} there\'s {self.counter} article related to the topic \n{self.sep_sign * 90}')
        elif self.counter == 0:
            print(f'{self.sep_sign * 90}\n On the page {self.n_page} there\'s no articles related to the topic \n{self.sep_sign * 90}')
        else:
            print(f'{self.sep_sign * 90}\n On the page {self.n_page} there\'re {self.counter} articles related to the topic \n{self.sep_sign * 90}')
    
    def selector (self, head, descr, ref):     
        for kw in self.list_of_kws: # iterating through the kwords
            if head not in self.controller: # check if the article has already been counted
                if kw in head.lower() or kw in descr.lower(): # matching keywords and heads or descriptions of the article
                    self.counter += 1
                    self.controller.append(head)
                    if self.show:
                        print('\n', self.counter, ') \t', head, '\n', descr, '\n', f'{self.root}{ref}', '\n')
                    if self.download:
                        self.article_downloader(head, ref)
            
    def related_articles_parser(self, show=True, total=None, download=None):
        """
        Launches parsing mechanisms iterating through the chosen range of the pages
        Launches additional methods - 'total' and 'show'
        """
        self.kw_definer()
        self.total_counter = 0
        self.show = show
        self.download = download
        for self.n_page in range(self.init_page, self.end_page+1):
            self.parser_engine()
            
        if total:
            print(f'\n In total there\'s {self.total_counter} related articles in {self.end_page} pages of the Economist "{self.section}" section')

In [331]:
economist = economist_parser('https://www.economist.com', 'china', 1, 30)

In [332]:
economist.related_articles_parser(total=True, download=True)

Please enter the keywords, dividing them by a single comma: 
putin,russia,elon musk,   Putin
 On the page 1 there's no articles related to the topic 
 On the page 2 there's no articles related to the topic 

 1 ) 	 Xi Jinping won’t ditch Vladimir Putin, for now 
 China’s goal in Ukraine is Western disunity and failure, more than a Russian triumph 
 https://www.economist.com/china/2022/09/15/xi-jinping-wont-ditch-vladimir-putin-for-now 

******************** Download success ********************
 On the page 3 there's 1 article related to the topic 
 On the page 4 there's no articles related to the topic 
 On the page 5 there's no articles related to the topic 
 On the page 6 there's no articles related to the topic 
 On the page 7 there's no articles related to the topic 
 On the page 8 there's no articles related to the topic 

 1 ) 	 China unveils its vision of a global security order 
 Xi Jinping lays out a worldview with Chinese-Russian roots 
 https://www.economist.com/china/2022/

 On the page 21 there's no articles related to the topic 
 On the page 22 there's no articles related to the topic 
 On the page 23 there's no articles related to the topic 
 On the page 24 there's no articles related to the topic 
 On the page 25 there's no articles related to the topic 
 On the page 26 there's no articles related to the topic 
 On the page 27 there's no articles related to the topic 
 On the page 28 there's no articles related to the topic 
 On the page 29 there's no articles related to the topic 
 On the page 30 there's no articles related to the topic 

 In total there's 12 related articles in 30 pages of the Economist "china" section
